<a href="https://colab.research.google.com/github/shannonshih/Tibame_GAD245_Practice_GCP-BigQuery-python-code_0312/blob/main/BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

安裝所需的軟體包

In [ ]:
!pip install google-cloud-bigquery

設置客戶端

In [14]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad245-16-0305-96bb09dc9463.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')

connection done


建立資料集

In [15]:
# 定義資料集參數
dataset_id = 'tibame_gad245_16_0312_dataset_us'  # 替換為想要建立的資料集名稱
project_id = 'tibame-gad245-16-0305'  # 替換為 Google Cloud 專案 ID
dataset_ref = client.dataset(dataset_id)

# 設定資料集的描述和其他選項
dataset = bigquery.Dataset(dataset_ref)
dataset.description = 'This is a new dataset created from Python.'
dataset.location = 'US'  # 設定資料集的地區位置

# 建立資料集
dataset = client.create_dataset(dataset, timeout=30)  # 可設定超時時間
print(f"dataset: {dataset.dataset_id} ,created")

dataset: tibame_gad245_16_0312_dataset_us ,created


建立資料表

In [3]:
# 定義資料集和資料表
# dataset_id = '資料集ID'
table_id = 'age_table'
schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
]

# 定義表格參數
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# 建立資料表
table = client.create_table(table)
print(f"table: {table.table_id} ,created")

table: age_table ,created


寫入資料

In [4]:
# 定義資料集和表資料表名稱
# dataset_id = '資料集ID'
table_id = 'age_table'

# 定義資料
rows_to_insert = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 35},
    {"name": "Shannon", "age": 27}
]

# 插入資料
errors = client.insert_rows_json(f"{dataset_id}.{table_id}", rows_to_insert)

if errors == []:
    print("data insert successfully")
else:
    print(f"error: {errors}")



data insert successfully


查詢資料

In [8]:
# 定義查詢語句
query = """
    SELECT name, age
    FROM `tibame_gad245_16_0312_dataset_us.age_table`
    WHERE age > 30
"""

# 執行查詢
query_job = client.query(query)

# 獲取結果
results = query_job.result()

# 印出結果
for row in results:
    print(f"name: {row.name}, age: {row.age}")

print(results)

name: Bob, age: 35


刪除資料表

In [9]:
# 定義資料集和資料表名稱
# dataset_id = '資料集ID'
# table_id = 'age_table'

# 定義資料表參數
table_ref = client.dataset(dataset_id).table(table_id)

# 刪除資料表
client.delete_table(table_ref)
print(f"table: {table_id} ,deleted")


table: age_table ,deleted


建立tenserflow模型

In [16]:
# 定義 SQL 查詢以創建模型
query = """
CREATE OR REPLACE MODEL `tibame_gad245_16_0312_dataset_us.imported_tf_model`
OPTIONS (MODEL_TYPE='TENSORFLOW',
         MODEL_PATH='gs://cloud-training-demos/txtclass/export/exporter/1549825580/*')
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("TensorFlow model import BigQuery ML")

TensorFlow model import BigQuery ML


測試tensorflow模型資料

In [17]:
# 定義 SQL 查詢以使用模型進行預測，只取五筆非 NULL 的資料
query = """
SELECT *
FROM ML.PREDICT(MODEL `tibame_gad245_16_0312_dataset_us.imported_tf_model`,
  (SELECT title AS input
   FROM `bigquery-public-data.hacker_news.full`
   WHERE title IS NOT NULL
   LIMIT 5)  -- 限制結果為五筆資料
)
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)

Row(([0.8007680773735046, 0.10563275963068008, 0.09359916299581528], 'Placeholder'), {'dense_1': 0, 'input': 1})
Row(([0.8007680773735046, 0.10563275963068008, 0.09359916299581528], 'Search Engine Optimization'), {'dense_1': 0, 'input': 1})
Row(([0.00046904210466891527, 0.9701762795448303, 0.029354579746723175], 'Reduce Bounce Rate of Your Web Pages – Imprace Improves the First Impression'), {'dense_1': 0, 'input': 1})
Row(([0.029970815405249596, 0.9580219388008118, 0.012007219716906548], 'Harlequin Presents : Hidden Object of Desire Game Download Free'), {'dense_1': 0, 'input': 1})
Row(([3.218979429675528e-07, 2.498678077245131e-05, 0.9999747276306152], 'Is it true Nisim.com coupon or coupon codes really reduce amount from purchase?'), {'dense_1': 0, 'input': 1})


刪除資料集

In [18]:
# 定義資料集名稱
dataset_id = 'tibame_gad245_16_0312_dataset_us'

# 定義資料集參數
dataset_ref = client.dataset(dataset_id)

# 刪除資料集
client.delete_dataset(dataset_ref, delete_contents=True)
print(f"dataset: {dataset_id} ,deleted")

dataset: tibame_gad245_16_0312_dataset_us ,deleted
